In [1]:
headers_str = """Accept: application/json, text/plain, */*
Accept-Encoding: gzip, deflate, br
Accept-Language: pt-BR,pt;q=0.9,en;q=0.8,es;q=0.7,it;q=0.6
ActivityId: c339cebe-0a40-464b-a762-e13c2b8d6de9
Connection: keep-alive
Content-Length: 1245
Content-Type: application/json;charset=UTF-8
Host: wabi-brazil-south-b-primary-api.analysis.windows.net
Origin: https://app.powerbi.com
Referer: https://app.powerbi.com/
RequestId: 3a12cd4b-59e2-33f5-9736-4e183bcc2b97
Sec-Fetch-Dest: empty
Sec-Fetch-Mode: cors
Sec-Fetch-Site: cross-site
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36
X-PowerBI-ResourceKey: fc7e5364-ba9d-4cf2-a1b4-0b36c1abc7cd
sec-ch-ua: "Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"
sec-ch-ua-mobile: ?0
sec-ch-ua-platform: "macOS" """

dict_headers = {}
for line in headers_str.split("\n"):
    item = line.split(": ")
    dict_headers[item[0]] = item[1]

In [2]:
import requests
import json

def get_json_req(city, state, year):

    json_data = {"version":"1.0.0","queries":
                 [{"Query":{"Commands":
                            [{"SemanticQueryDataShapeCommand":
                              {"Query":{"Version":2,
                                        "From":[{"Name":"v","Entity":"Valores Gerais 2018 a 2022","Type":0},
                                                {"Name":"d","Entity":"disp2023","Type":0}],
                                        "Select":[{"Measure":{"Expression":{"SourceRef":{"Source":"v"}},
                                                              "Property":f"{year}_pm1disp"},
                                                   "Name":"Valores Gerais 2018 a 2022.2022_pm1disp"},
                                                  {"Measure":{"Expression":{"SourceRef":{"Source":"v"}},
                                                              "Property":f"{year}_EmPrEP"},
                                                   "Name":"Valores Gerais 2018 a 2022.2022_EmPrEP"},
                                                  {"Measure":{"Expression":{"SourceRef":{"Source":"v"}},
                                                              "Property":f"{year}_Descont"},
                                                   "Name":"Valores Gerais 2018 a 2022.2022_Descont"}],
                                        "Where":[{"Condition":{"In":{"Expressions":[{"Column":{"Expression":
                                                                                               {"SourceRef":{"Source":"d"}},
                                                                                               "Property":"UnidadeFederada"}},
                                                                                    {"Column":{"Expression":{"SourceRef":{"Source":"d"}},
                                                                                               "Property":"nome_município"}}],
                                                                     "Values":[[{"Literal":{"Value": f"'{state}'"}},
                                                                                {"Literal":{"Value":f"'{city}'"}}]]}}}]},
                               "Binding":{"Primary":{"Groupings":[{"Projections":[0,1,2]}]},"Version":1},
                               "ExecutionMetricsKind":1}}]},"QueryId":"",
                   "ApplicationContext":{"DatasetId":"5f667bf6-c58b-4dee-b0ac-5af356a8e217",
                                         "Sources":[{"ReportId":"40134fda-81f3-4f99-99ec-b497897bf920",
                                                     "VisualId":"a809ec7d5188ab673473"}]}}],
                 "cancelQueries":[],"modelId":1444769}
    return json_data

def send_request(city, state, year):
    request_body = get_json_req(city, state, year)
    url = "https://wabi-brazil-south-b-primary-api.analysis.windows.net/public/reports/querydata?synchronous=true"
    r = requests.post(url, data=json.dumps(request_body), headers=dict_headers)
    return r

In [3]:
get_json_req(city, state, year)

NameError: name 'city' is not defined

In [5]:
import pandas as pd
city_state = pd.read_csv("data/city_state.csv")

In [10]:
def fill_dict_response(c, city_code, dict_resp):
    dict_resp["city_code"].append(city_code)
    if len(c) == 0:
        dict_resp["started"].append(0)
        dict_resp["following"].append(0)
        dict_resp["loss_of_follow"].append(0)

    if len(c) == 2:
        dict_resp["started"].append(c[0]) 
        dict_resp["following"].append(c[1]) 
        dict_resp["loss_of_follow"].append(0) 
    
    if len(c) == 3:
        dict_resp["started"].append(c[0]) 
        dict_resp["following"].append(c[1]) 
        dict_resp["loss_of_follow"].append(c[2])
    return dict_resp

In [57]:
import numpy as np

year = 2018
array_2018 = np.array([])

import time

dataset = city_state.query("cases > 5")


for year in range(2018, 2023):
    for i, row in dataset.sample(frac=0.03).iterrows():
        dict_year = {"city_code":[], "started":[],
             "following":[], "loss_of_follow":[]}

        print(i)
        time.sleep(0.3)
        city = row["city_name"]
        state = row["state_name"]

        if "\'" in city:
            print(f"city not requested: {city}")
            continue

        response = send_request(city, state, year)

        c = response.json()["results"][0]["result"]["data"]["dsr"]["DS"][0]["PH"][0]["DM0"][0]["C"]
        city_code = row["city_code"]
        fill_dict_response(c, city_code, dict_year)

        pd.DataFrame(dict_year).to_csv(f"prep{year}_new.csv", mode="w", index="false")

4982
3132
1474
196
5384
659
1097
1757
4605
3649
160
179
3364
5087
3184
978
3981
1675
593
2981
5162
158
5505
3186
1337
4764
3215
1039
5067
3223
3560
5019
5069
3970
3434
541
3274
2513
3105
946
484
3208
5091
4164
376
4221
3416
1057
4960
1283
3416
4980
882
3125
1495
178
1896
3356
3522
5165
1162
664
2196
1057
5123
5240
4587
3454
2902
4337
3229
3364
5251
4120
931
3177
3811
1495
236
264
4931
749
541
4415
city not requested: Herval d'Oeste
187
664
918
5469
657
4578
3444
4436
3900
4768
3593
3478
2107
2137
500
3274
1620
3452
518
225
978
5469
322
3335
3345
3519
4298
1469
3207
5067
208
2918
3235
632
5615
169


#### test

In [15]:
request_body = get_json_req("São Paulo", "São Paulo", 2020)
url = "https://wabi-brazil-south-b-primary-api.analysis.windows.net/public/reports/querydata?synchronous=true"
r = requests.post(url, data=json.dumps(request_body), headers=dict_headers)
r.text
#r.json()["results"][0]["result"]["data"]["dsr"]["DS"][0]["PH"][0]["DM0"][0]#["C"]

'{"jobIds":["232eca72-98c6-4728-a28f-0643cc757751"],"results":[{"jobId":"232eca72-98c6-4728-a28f-0643cc757751","result":{"data":{"timestamp":"2023-05-01T09:36:29.362Z","rootActivityId":"70d1a80a-74cc-4c79-bd77-47e252075350","descriptor":{"Select":[{"Kind":2,"Value":"M0","Format":"#,0","Name":"Valores Gerais 2018 a 2022.2022_pm1disp"},{"Kind":2,"Value":"M1","Format":"#,0","Name":"Valores Gerais 2018 a 2022.2022_EmPrEP"},{"Kind":2,"Value":"M2","Format":"#,0","Name":"Valores Gerais 2018 a 2022.2022_Descont"}],"Version":2},"metrics":{"Version":"1.0.0","Events":[{"Id":"c0b79f6e-c851-4988-a88f-8b13c47d900b","Name":"Execute Semantic Query","Component":"DSE","Start":"2023-05-01T09:36:29.3625867Z","End":"2023-05-01T09:36:29.5657098Z"},{"Id":"17509647-9978-4400-8338-558a501dbcf8","ParentId":"c0b79f6e-c851-4988-a88f-8b13c47d900b","Name":"Execute DAX Query","Component":"DSE","Start":"2023-05-01T09:36:29.3625867Z","End":"2023-05-01T09:36:29.5657098Z","Metrics":{"RowCount":1}},{"Id":"2019A5A9-B6E7-4

In [8]:
city_state.query("city_name == 'São Gonçalo do Amarante' ")

,Unnamed: 0,city_code,city_name,year,cases,group,region_type,state_code,state_name
1052,1052,231240,São Gonçalo do Amarante,2018,8,total_hiv,city,23,Ceará
1205,1205,241200,São Gonçalo do Amarante,2018,21,total_hiv,city,24,Rio Grande do Norte
